# Colab inference for ZFTurbo's [Music-Source-Separation-Training](https://github.com/ZFTurbo/Music-Source-Separation-Training/)


<font size=1>*made by [jarredou](https://github.com/jarredou) & deton</font>  
[![ko-fi](https://ko-fi.com/img/githubbutton_sm.svg)](https://ko-fi.com/Q5Q811R5YI)

<font size=1>Visit [models list](https://docs.google.com/document/d/17fjNvJzj8ZGSer7c7OFe_CNfUKbAxEh_OBv94ZdRG5c/edit?tab=t.0#heading=h.2vdz5zlpb27h) for their descriptions</font>  

In [ ]:
#@markdown #GDrive connection
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import base64
#@markdown # Install

%cd /content
!git clone -b colab-inference https://github.com/deton24/Music-Source-Separation-Training-Colab-2025/

#requirements fix by santilli_
req_text = """
mutagen==1.47.0
ml_collections==1.1.0
numpy>=1.26.0
pandas==2.2.2
scipy
tqdm
segmentation_models_pytorch==0.3.3
timm
audiomentations
pedalboard
omegaconf
beartype
rotary_embedding_torch==0.3.5
einops
# librosa==0.11.0
demucs #==4.0.0
# transformers==4.35.0
torchmetrics==0.11.4
spafe==0.3.2
protobuf
torch_audiomentations
asteroid==0.7.0
auraloss
torchseg
"""

with open("Music-Source-Separation-Training/requirements.txt", "w") as f:
    f.write(req_text)

!mkdir '/content/Music-Source-Separation-Training/ckpts'

print('Installing the dependencies... This will take a few minutes')
!pip install -r 'Music-Source-Separation-Training/requirements.txt' &> /dev/null

print('Installation is done !')

In [ ]:
%cd '/content/Music-Source-Separation-Training/'
import os
import torch
import yaml
from urllib.parse import quote

class IndentDumper(yaml.Dumper):
    def increase_indent(self, flow=False, indentless=False):
        return super(IndentDumper, self).increase_indent(flow, False)


def tuple_constructor(loader, node):
    # Load the sequence of values from the YAML node
    values = loader.construct_sequence(node)
    # Return a tuple constructed from the sequence
    return tuple(values)

# Register the constructor with PyYAML
yaml.SafeLoader.add_constructor('tag:yaml.org,2002:python/tuple',
tuple_constructor)



def conf_edit(config_path, chunk_size, overlap):
    with open(config_path, 'r') as f:
        data = yaml.load(f, Loader=yaml.SafeLoader)

    # handle cases where 'use_amp' is missing from config:
    if 'use_amp' not in data.keys():
      data['training']['use_amp'] = True

    data['audio']['chunk_size'] = chunk_size
    data['inference']['num_overlap'] = overlap

    if data['inference']['batch_size'] == 1:
      data['inference']['batch_size'] = 2

    print("Using custom overlap and chunk_size values:")
    print(f"overlap = {data['inference']['num_overlap']}")
    print(f"chunk_size = {data['audio']['chunk_size']}")
    print(f"batch_size = {data['inference']['batch_size']}")

    with open(config_path, 'w') as f:
        yaml.dump(data, f, default_flow_style=False, sort_keys=False, Dumper=IndentDumper, allow_unicode=True)

def download_file(url):
    # Encode the URL to handle spaces and special characters
    encoded_url = quote(url, safe=':/')

    path = 'ckpts'
    os.makedirs(path, exist_ok=True)
    filename = os.path.basename(encoded_url)
    file_path = os.path.join(path, filename)

    if os.path.exists(file_path):
        print(f"File '{filename}' already exists at '{path}'.")
        return

    try:
        response = torch.hub.download_url_to_file(encoded_url, file_path)
        print(f"File '{filename}' downloaded successfully")
    except Exception as e:
        print(f"Error downloading file '{filename}' from '{url}': {e}")





#@markdown # Separation
#@markdown #### Separation config:
input_folder = '/content/drive/MyDrive/input' #@param {type:"string"}
output_folder = '/content/drive/MyDrive/output' #@param {type:"string"}
model = 'VOCALS-MelBand-Roformer (by KimberleyJSN)' #@param ['VOCALS-MelBand-Roformer (by KimberleyJSN)', 'VOCALS-MelBand-Roformer (by Becruily)', 'VOCALS-MelBand-Roformer voc_Fv5 (by Gabox)', 'VOCALS-MelBand-Roformer voc_Fv4 (by Gabox)', 'VOCALS-MelBand-Roformer voc_Fv6 experimental (by Gabox)', 'VOCALS-MelBand-Roformer voc_Fv3 (by Gabox)', 'VOCALS-Melband-Roformer BigBeta6 (by unwa)', 'VOCALS-Melband-Roformer BigBeta6X (by unwa)', 'VOCALS-MelBand-Roformer voc_gabox2 (by Gabox)', 'VOCALS-MelBand-Roformer Kim FT 2 Bleedless (by Unwa)', 'VOCALS-MelBand-Roformer Kim FT 2 (by Unwa)', 'VOCALS-Mel-Roformer FT 3 Preview (by unwa)', 'VOCALS-MelBand-Roformer Kim FT (by Unwa)', 'VOCALS-Melband-Roformer BigBeta5e (by unwa)', 'VOCALS-Mel-Roformer BigBeta4 (by unwa)', 'VOCALS-BS-Roformer Resurrection (by unwa)', 'VOCALS-BS-Roformer Revive 3e (by unwa)', 'VOCALS-BS-Roformer Revive 2 (by unwa)', 'VOCALS-BS-Roformer Revive (by unwa)', 'VOCALS-BS-RoformerLargev1 (by unwa)', 'VOCALS-BS-Roformer_1297 (by viperx)', 'VOCALS-BS-Roformer_1296 (by viperx)', 'VOCALS-InstVocHQ', 'INST-Mel-Roformer v1e (by unwa)', 'INST-Mel-Roformer v1e+ (by unwa)', 'INST-BS-Roformer Resurrection (by unwa)', 'INST-Mel-Roformer INSTV8B (by Gabox)', 'INST-MelBand-Roformer Inst_Fv8 v2 (by Gabox)', 'INST-MelBand-Roformer (by Becruily)', 'INST-Mel-Roformer INSTV7 (by Gabox)', 'INST-MelBand-Roformer Inst_Fv4 (by Gabox)','INST-Mel-Roformer Neo_InstVFX (by neoculture)', 'INST-Mel-Roformer INSTFVX (by Gabox)', 'INST-Mel-Roformer INSTV7N (by Gabox)', 'INST-Mel-Roformer INSTFV7Z (by Gabox)', 'INST-Mel-Roformer INSTV6N (by Gabox)', 'INST-Mel-Roformer INSTV5 (by Gabox)', 'INST-Mel-Roformer INSTV6 (by Gabox)', 'INST-Mel-Roformer inst_gabox3 (by Gabox)', 'INST-Mel-Roformer Metal Model Preview (by Mesk)', 'INST-VOC-Mel-Roformer a.k.a. duality (by unwa)', 'INST-VOC-Mel-Roformer a.k.a. duality v2 (by unwa)', 'INST-Mel-Roformer v1 (by unwa)', 'INST-Mel-Roformer v1+ (by unwa)', 'INST-Mel-Roformer v2 (by unwa)', 'KARAOKE-MelBand-Roformer (by becruily)', 'KARAOKE-MelBand-Roformer (by aufr33 & viperx)', 'OTHER-BS-Roformer_1053 (by viperx)',  '6STEMS-BS-Roformer-SW','4STEMS-MelBand-Roformer_Large (by Amane)', '4STEMS-SCNet_XL_MUSDB18 (by ZFTurbo)', '4STEMS-SCNet_Large (by starrytong)', '4STEMS-BS-Roformer_MUSDB18 (by ZFTurbo)', '4STEMS-SCNet_MUSDB18 (by starrytong)', 'CROWD-REMOVAL-MelBand-Roformer (by aufr33)', 'VOCALS-VitLarge23 (by ZFTurbo)', 'CINEMATIC-BandIt_Plus (by kwatcharasupat)', 'CINEMATIC-BandIt_v2 multi (by kwatcharasupat)', 'DRUMSEP-MDX23C_DrumSep_5stem_new (by jarredou)', 'DRUMSEP-MDX23C_DrumSep_6stem (by aufr33 & jarredou)', 'DE-REVERB-MDX23C (by aufr33 & jarredou)', 'DE-REVERB-MelBand-Roformer aggr./v2/19.1729 (by anvuew)', 'DE-REVERB-BS-Roformer dereverb_room mono (by anvuew)', 'DE-REVERB-MelBand-Roformer mono 20.4029 (by anvuew)', 'DE-REVERB-Echo-MelBand-Roformer (by Sucial)', 'LEAD-VOCAL-DE-REVERB-MelBand-Roformer (by Gabox)', 'DENOISE-MelBand-Roformer-1 (by aufr33)', 'DENOISE-MelBand-Roformer-2 (by aufr33)', 'DEBLEED-MelBand-Roformer (by unwa/97chris)', 'DENOISE-DEBLEED-MelBand-Roformer (by Gabox)', 'VOCALS-Male Female-BS-RoFormer Male Female Beta 7_2889 (by aufr33)', 'PHANTOM-CENTER-HTDemucs (by wesleyr36)', 'GUITAR-MelBand-Roformer (by becruily)']
extract_instrumental = True #@param {type:"boolean"}
export_format = 'flac PCM_16' #@param ['wav FLOAT', 'flac PCM_16', 'flac PCM_24']
use_tta = False #@param {type:"boolean"}
#@markdown ---
#@markdown *Roformers custom config:*
overlap = 2 #@param {type:"slider", min:2, max:40, step:1}
chunk_size = "485100" #@param [88200, 112455, 132300, 156555, 176400, 352800, 485100, 529200, 588800, 587412, 661500, 749259] {allow-input: true}
chunk_size = int(chunk_size)

if export_format.startswith('flac'):
    flac_file = True
    pcm_type = export_format.split(' ')[1]
else:
    flac_file = False
    pcm_type = None

if model == 'VOCALS-InstVocHQ':
    model_type = 'mdx23c'
    config_path = 'ckpts/config_vocals_mdx23c.yaml'
    start_check_point = 'ckpts/model_vocals_mdx23c_sdr_10.17.ckpt'
    download_file('https://raw.githubusercontent.com/ZFTurbo/Music-Source-Separation-Training/main/configs/config_vocals_mdx23c.yaml')
    download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v1.0.0/model_vocals_mdx23c_sdr_10.17.ckpt')

elif model == 'VOCALS-MelBand-Roformer (by Becruily)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_vocals_becruily.yaml'
    start_check_point = 'ckpts/mel_band_roformer_vocals_becruily.ckpt'
    download_file('https://huggingface.co/becruily/mel-band-roformer-vocals/resolve/main/config_vocals_becruily.yaml')
    download_file('https://huggingface.co/becruily/mel-band-roformer-vocals/resolve/main/mel_band_roformer_vocals_becruily.ckpt')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'INST-MelBand-Roformer (by Becruily)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_instrumental_becruily.yaml'
    start_check_point = 'ckpts/mel_band_roformer_instrumental_becruily.ckpt'
    download_file('https://huggingface.co/becruily/mel-band-roformer-instrumental/resolve/main/config_instrumental_becruily.yaml')
    download_file('https://huggingface.co/becruily/mel-band-roformer-instrumental/resolve/main/mel_band_roformer_instrumental_becruily.ckpt')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-MelBand-Roformer (by KimberleyJSN)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_vocals_mel_band_roformer_kj.yaml'
    start_check_point = 'ckpts/MelBandRoformer.ckpt'
    download_file('https://raw.githubusercontent.com/ZFTurbo/Music-Source-Separation-Training/main/configs/KimberleyJensen/config_vocals_mel_band_roformer_kj.yaml')
    download_file('https://huggingface.co/KimberleyJSN/melbandroformer/resolve/main/MelBandRoformer.ckpt')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-MelBand-Roformer voc_Fv5 (by Gabox)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/voc_gabox.yaml'
    start_check_point = 'ckpts/voc_fv5.ckpt'
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/vocals/voc_fv5.ckpt')
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/vocals/voc_gabox.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-MelBand-Roformer voc_gabox2 (by Gabox)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/voc_gabox.yaml'
    start_check_point = 'ckpts/voc_gabox2.ckpt'
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/vocals/voc_gabox2.ckpt')
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/vocals/voc_gabox.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-MelBand-Roformer voc_Fv4 (by Gabox)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/voc_gabox.yaml'
    start_check_point = 'ckpts/voc_fv4.ckpt'
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/vocals/voc_gabox.yaml')
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/vocals/voc_fv4.ckpt')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-MelBand-Roformer voc_Fv6 experimental (by Gabox)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/voc_gabox.yaml'
    start_check_point = 'ckpts/voc_fv6.ckpt'
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/vocals/voc_gabox.yaml')
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/vocals/voc_fv6.ckpt')
    conf_edit(config_path, chunk_size, overlap)


elif model == 'VOCALS-MelBand-Roformer voc_Fv3 (by Gabox)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/voc_gabox.yaml'
    start_check_point = 'ckpts/voc_Fv3.ckpt'
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/vocals/voc_gabox.yaml')
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/vocals/voc_Fv3.ckpt')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-MelBand-Roformer Kim FT (by Unwa)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_kimmel_unwa_ft.yaml'
    start_check_point = 'ckpts/kimmel_unwa_ft.ckpt'
    download_file('https://huggingface.co/pcunwa/Kim-Mel-Band-Roformer-FT/resolve/main/config_kimmel_unwa_ft.yaml')
    download_file('https://huggingface.co/pcunwa/Kim-Mel-Band-Roformer-FT/resolve/main/kimmel_unwa_ft.ckpt')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-MelBand-Roformer Kim FT 2 (by Unwa)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_kimmel_unwa_ft.yaml'
    start_check_point = 'ckpts/kimmel_unwa_ft.ckpt'
    download_file('https://huggingface.co/pcunwa/Kim-Mel-Band-Roformer-FT/resolve/main/config_kimmel_unwa_ft.yaml')
    download_file('https://huggingface.co/pcunwa/Kim-Mel-Band-Roformer-FT/resolve/main/kimmel_unwa_ft2.ckpt')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-MelBand-Roformer Kim FT 2 Bleedless (by Unwa)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_kimmel_unwa_ft.yaml'
    start_check_point = 'ckpts/kimmel_unwa_ft2_bleedless.ckpt'
    download_file('https://huggingface.co/pcunwa/Kim-Mel-Band-Roformer-FT/resolve/main/config_kimmel_unwa_ft.yaml')
    download_file('https://huggingface.co/pcunwa/Kim-Mel-Band-Roformer-FT/resolve/main/kimmel_unwa_ft2_bleedless.ckpt')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-Mel-Roformer FT 3 Preview (by unwa)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_kimmel_unwa_ft.yaml'
    start_check_point = 'ckpts/kimmel_unwa_ft3_prev.ckpt'
    download_file('https://huggingface.co/pcunwa/Kim-Mel-Band-Roformer-FT/resolve/main/kimmel_unwa_ft3_prev.ckpt')
    download_file('https://huggingface.co/pcunwa/Kim-Mel-Band-Roformer-FT/resolve/main/config_kimmel_unwa_ft.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-BS-Roformer_1297 (by viperx)':
    model_type = 'bs_roformer'
    config_path = 'ckpts/model_bs_roformer_ep_317_sdr_12.9755.yaml'
    start_check_point = 'ckpts/model_bs_roformer_ep_317_sdr_12.9755.ckpt'
    download_file('https://raw.githubusercontent.com/ZFTurbo/Music-Source-Separation-Training/main/configs/viperx/model_bs_roformer_ep_317_sdr_12.9755.yaml')
    download_file('https://github.com/TRvlvr/model_repo/releases/download/all_public_uvr_models/model_bs_roformer_ep_317_sdr_12.9755.ckpt')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-BS-Roformer_1296 (by viperx)':
    model_type = 'bs_roformer'
    config_path = 'ckpts/model_bs_roformer_ep_368_sdr_12.9628.yaml'
    start_check_point = 'ckpts/model_bs_roformer_ep_368_sdr_12.9628.ckpt'
    download_file('https://github.com/TRvlvr/model_repo/releases/download/all_public_uvr_models/model_bs_roformer_ep_368_sdr_12.9628.ckpt')
    download_file('https://raw.githubusercontent.com/TRvlvr/application_data/main/mdx_model_data/mdx_c_configs/model_bs_roformer_ep_368_sdr_12.9628.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-BS-RoformerLargev1 (by unwa)':
    model_type = 'bs_roformer'
    config_path = 'ckpts/config_bsrofoL.yaml'
    start_check_point = 'ckpts/BS-Roformer_LargeV1.ckpt'
    download_file('https://huggingface.co/jarredou/unwa_bs_roformer/resolve/main/BS-Roformer_LargeV1.ckpt')
    download_file('https://huggingface.co/jarredou/unwa_bs_roformer/raw/main/config_bsrofoL.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-BS-Roformer Resurrection (by unwa)':
    model_type = 'bs_roformer'
    config_path = 'ckpts/BS-Roformer-Resurrection-Config.yaml'
    start_check_point = 'ckpts/BS-Roformer-Resurrection.ckpt'
    download_file('https://huggingface.co/pcunwa/BS-Roformer-Resurrection/resolve/main/BS-Roformer-Resurrection.ckpt')
    download_file('https://huggingface.co/pcunwa/BS-Roformer-Resurrection/resolve/main/BS-Roformer-Resurrection-Config.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-BS-Roformer Revive (by unwa)':
    model_type = 'bs_roformer'
    config_path = 'ckpts/config.yaml'
    start_check_point = 'ckpts/bs_roformer_revive.ckpt'
    download_file('https://huggingface.co/pcunwa/BS-Roformer-Revive/resolve/main/bs_roformer_revive.ckpt')
    download_file('https://huggingface.co/pcunwa/BS-Roformer-Revive/resolve/main/config.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-BS-Roformer Revive 2 (by unwa)':
    model_type = 'bs_roformer'
    config_path = 'ckpts/config.yaml'
    start_check_point = 'ckpts/bs_roformer_revive2.ckpt'
    download_file('https://huggingface.co/pcunwa/BS-Roformer-Revive/resolve/main/bs_roformer_revive2.ckpt')
    download_file('https://huggingface.co/pcunwa/BS-Roformer-Revive/resolve/main/config.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-BS-Roformer Revive 3e (by unwa)':
    model_type = 'bs_roformer'
    config_path = 'ckpts/config.yaml'
    start_check_point = 'ckpts/bs_roformer_revive3e.ckpt'
    download_file('https://huggingface.co/pcunwa/BS-Roformer-Revive/resolve/main/bs_roformer_revive3e.ckpt')
    download_file('https://huggingface.co/pcunwa/BS-Roformer-Revive/resolve/main/config.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-Melband-Roformer BigBeta6X (by unwa)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/big_beta6x.yaml'
    start_check_point = 'ckpts/big_beta6x.ckpt'
    download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-big/resolve/main/big_beta6x.ckpt')
    download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-big/resolve/main/big_beta6x.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-Melband-Roformer BigBeta6 (by unwa)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/big_beta6.yaml'
    start_check_point = 'ckpts/big_beta6.ckpt'
    download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-big/resolve/main/big_beta6.ckpt')
    download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-big/resolve/main/big_beta6.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-Melband-Roformer BigBeta5e (by unwa)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/big_beta5e.yaml'
    start_check_point = 'ckpts/big_beta5e.ckpt'
    download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-big/resolve/main/big_beta5e.ckpt')
    download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-big/resolve/main/big_beta5e.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-Mel-Roformer BigBeta4 (by unwa)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_melbandroformer_big_beta4.yaml'
    start_check_point = 'ckpts/melband_roformer_big_beta4.ckpt'
    download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-big/resolve/main/melband_roformer_big_beta4.ckpt')
    download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-big/raw/main/config_melbandroformer_big_beta4.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'INST-Mel-Roformer v1 (by unwa)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_melbandroformer_inst.yaml'
    start_check_point = 'ckpts/melband_roformer_inst_v1.ckpt'
    download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-Inst/resolve/main/melband_roformer_inst_v1.ckpt')
    download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-Inst/raw/main/config_melbandroformer_inst.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'INST-Mel-Roformer v2 (by unwa)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_melbandroformer_inst_v2.yaml'
    start_check_point = 'ckpts/melband_roformer_inst_v2.ckpt'
    download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-Inst/resolve/main/melband_roformer_inst_v2.ckpt')
    download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-Inst/raw/main/config_melbandroformer_inst_v2.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'INST-Mel-Roformer v1e (by unwa)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_melbandroformer_inst.yaml'
    start_check_point = 'ckpts/inst_v1e.ckpt'
    download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-Inst/resolve/main/inst_v1e.ckpt')
    download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-Inst/raw/main/config_melbandroformer_inst.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'INST-BS-Roformer Resurrection (by unwa)':
    model_type = 'bs_roformer'
    config_path = 'ckpts/BS-Roformer-Resurrection-Inst-Config.yaml'
    start_check_point = 'ckpts/BS-Roformer-Resurrection-Inst.ckpt'
    download_file('https://huggingface.co/pcunwa/BS-Roformer-Resurrection/resolve/main/BS-Roformer-Resurrection-Inst.ckpt')
    download_file('https://huggingface.co/pcunwa/BS-Roformer-Resurrection/resolve/main/BS-Roformer-Resurrection-Inst-Config.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'INST-Mel-Roformer INSTV8B (by Gabox)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/inst_gabox.yaml'
    start_check_point = 'ckpts/Inst_FV8b.ckpt'
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/experimental/Inst_FV8b.ckpt')
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/instrumental/inst_gabox.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'INST-Mel-Roformer INSTV7 (by Gabox)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/inst_gabox.yaml'
    start_check_point = 'ckpts/Inst_GaboxV7.ckpt'
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/instrumental/Inst_GaboxV7.ckpt')
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/instrumental/inst_gabox.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'INST-Mel-Roformer v1e+ (by unwa)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_melbandroformer_inst.yaml'
    start_check_point = 'ckpts/inst_v1e_plus.ckpt'
    download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-Inst/resolve/main/inst_v1e_plus.ckpt')
    download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-Inst/resolve/main/config_melbandroformer_inst.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'INST-MelBand-Roformer Inst_Fv8 v2 (by Gabox)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/inst_gabox.yaml'
    start_check_point = 'ckpts/Inst_GaboxFv8.ckpt'
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/instrumental/Inst_GaboxFv8.ckpt')
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/instrumental/inst_gabox.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'INST-MelBand-Roformer Inst_Fv4 (by Gabox)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/inst_gabox.yaml'
    start_check_point = 'ckpts/Inst_GaboxFv4.ckpt'
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/instrumental/inst_Fv4.ckpt')
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/instrumental/inst_gabox.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'INST-Mel-Roformer INSTFVX (by Gabox)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/inst_gabox.yaml'
    start_check_point = 'ckpts/Inst_GaboxFVX.ckpt'
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/instrumental/Inst_GaboxFVX.ckpt')
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/instrumental/inst_gabox.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'INST-Mel-Roformer Neo_InstVFX (by neoculture)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_neo_inst.yaml'
    start_check_point = 'ckpts/Neo_InstVFX.ckpt'
    download_file('https://huggingface.co/natanworkspace/melband_roformer/resolve/main/Neo_InstVFX.ckpt')
    download_file('https://huggingface.co/natanworkspace/melband_roformer/resolve/main/config_neo_inst.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'INST-Mel-Roformer INSTFV7Z (by Gabox)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/inst_gabox.yaml'
    start_check_point = 'ckpts/Inst_GaboxFv7z.ckpt'
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/instrumental/Inst_GaboxFv7z.ckpt')
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/instrumental/inst_gabox.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'INST-Mel-Roformer INSTV7N (by Gabox)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/inst_gabox.yaml'
    start_check_point = 'ckpts/INSTV7N.ckpt'
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/instrumental/INSTV7N.ckpt')
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/instrumental/inst_gabox.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'INST-Mel-Roformer INSTV6N (by Gabox)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/inst_gabox.yaml'
    start_check_point = 'ckpts/INSTV6N.ckpt'
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/instrumental/INSTV6N.ckpt')
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/instrumental/inst_gabox.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'INST-Mel-Roformer v1+ (by unwa)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_melbandroformer_inst.yaml'
    start_check_point = 'ckpts/inst_v1_plus_test.ckpt'
    download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-Inst/resolve/main/inst_v1_plus_test.ckpt')
    download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-Inst/resolve/main/config_melbandroformer_inst.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'INST-Mel-Roformer INSTV5 (by Gabox)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/inst_gabox.yaml'
    start_check_point = 'ckpts/INSTV5.ckpt'
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/instrumental/INSTV5.ckpt')
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/instrumental/inst_gabox.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'INST-Mel-Roformer INSTV6 (by Gabox)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/inst_gabox.yaml'
    start_check_point = 'ckpts/INSTV6.ckpt'
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/instrumental/INSTV6.ckpt')
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/instrumental/inst_gabox.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'INST-Mel-Roformer inst_gabox3 (by Gabox)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/inst_gabox.yaml'
    start_check_point = 'ckpts/inst_gabox3.ckpt'
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/instrumental/inst_gabox3.ckpt')
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/instrumental/inst_gabox.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'INST-Mel-Roformer Metal Model Preview (by Mesk)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_inst_metal_roformer_mesk.yaml'
    start_check_point = 'ckpts/metal_roformer_inst_mesk_preview.ckpt'
    download_file('https://huggingface.co/meskvlla33/metal_roformer_preview/resolve/main/metal_roformer_inst_mesk_preview.ckpt')
    download_file('https://huggingface.co/meskvlla33/metal_roformer_preview/resolve/main/config_inst_metal_roformer_mesk.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'INST-VOC-Mel-Roformer a.k.a. duality (by unwa)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_melbandroformer_instvoc_duality.yaml'
    start_check_point = 'ckpts/melband_roformer_instvoc_duality_v1.ckpt'
    download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-InstVoc-Duality/resolve/main/melband_roformer_instvoc_duality_v1.ckpt')
    download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-InstVoc-Duality/raw/main/config_melbandroformer_instvoc_duality.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'INST-VOC-Mel-Roformer a.k.a. duality v2 (by unwa)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_melbandroformer_instvoc_duality.yaml'
    start_check_point = 'ckpts/melband_roformer_instvox_duality_v2.ckpt'
    download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-InstVoc-Duality/resolve/main/melband_roformer_instvox_duality_v2.ckpt')
    download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-InstVoc-Duality/raw/main/config_melbandroformer_instvoc_duality.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'KARAOKE-MelBand-Roformer (by becruily)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_karaoke_becruily.yaml'
    start_check_point = 'ckpts/mel_band_roformer_karaoke_becruily.ckpt'
    download_file('https://huggingface.co/becruily/mel-band-roformer-karaoke/resolve/main/mel_band_roformer_karaoke_becruily.ckpt')
    download_file('https://huggingface.co/becruily/mel-band-roformer-karaoke/resolve/main/config_karaoke_becruily.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'KARAOKE-MelBand-Roformer (by aufr33 & viperx)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_mel_band_roformer_karaoke.yaml'
    start_check_point = 'ckpts/mel_band_roformer_karaoke_aufr33_viperx_sdr_10.1956.ckpt'
    download_file('https://huggingface.co/jarredou/aufr33-viperx-karaoke-melroformer-model/resolve/main/mel_band_roformer_karaoke_aufr33_viperx_sdr_10.1956.ckpt')
    download_file('https://huggingface.co/jarredou/aufr33-viperx-karaoke-melroformer-model/resolve/main/config_mel_band_roformer_karaoke.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'OTHER-BS-Roformer_1053 (by viperx)':
    model_type = 'bs_roformer'
    config_path = 'ckpts/model_bs_roformer_ep_937_sdr_10.5309.yaml'
    start_check_point = 'ckpts/model_bs_roformer_ep_937_sdr_10.5309.ckpt'
    download_file('https://github.com/TRvlvr/model_repo/releases/download/all_public_uvr_models/model_bs_roformer_ep_937_sdr_10.5309.ckpt')
    download_file('https://raw.githubusercontent.com/TRvlvr/application_data/main/mdx_model_data/mdx_c_configs/model_bs_roformer_ep_937_sdr_10.5309.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'CROWD-REMOVAL-MelBand-Roformer (by aufr33)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/model_mel_band_roformer_crowd.yaml'
    start_check_point = 'ckpts/mel_band_roformer_crowd_aufr33_viperx_sdr_8.7144.ckpt'
    download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v.1.0.4/mel_band_roformer_crowd_aufr33_viperx_sdr_8.7144.ckpt')
    download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v.1.0.4/model_mel_band_roformer_crowd.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-VitLarge23 (by ZFTurbo)':
    model_type = 'segm_models'
    config_path = 'ckpts/config_vocals_segm_models.yaml'
    start_check_point = 'ckpts/model_vocals_segm_models_sdr_9.77.ckpt'
    download_file('https://raw.githubusercontent.com/ZFTurbo/Music-Source-Separation-Training/refs/heads/main/configs/config_vocals_segm_models.yaml')
    download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v1.0.0/model_vocals_segm_models_sdr_9.77.ckpt')

elif model == 'CINEMATIC-BandIt_Plus (by kwatcharasupat)':
    model_type = 'bandit'
    config_path = 'ckpts/config_dnr_bandit_bsrnn_multi_mus64.yaml'
    start_check_point = 'ckpts/model_bandit_plus_dnr_sdr_11.47.chpt'
    download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v.1.0.3/config_dnr_bandit_bsrnn_multi_mus64.yaml')
    download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v.1.0.3/model_bandit_plus_dnr_sdr_11.47.chpt')

elif model == 'CINEMATIC-BandIt_v2 multi (by kwatcharasupat)':
    model_type = 'bandit_v2'
    config_path = 'ckpts/config_dnr_bandit_v2_mus64.yaml'
    start_check_point = 'ckpts/checkpoint-multi_state_dict.ckpt'
    download_file('https://raw.githubusercontent.com/ZFTurbo/Music-Source-Separation-Training/refs/heads/main/configs/config_dnr_bandit_v2_mus64.yaml')
    download_file('https://huggingface.co/jarredou/banditv2_state_dicts_only/resolve/main/checkpoint-multi_state_dict.ckpt')

elif model == 'DRUMSEP-MDX23C_DrumSep_5stem_new (by jarredou)':
    model_type = 'mdx23c'
    config_path = 'ckpts/config_mdx23c.yaml'
    start_check_point = 'ckpts/drumsep_5stems_mdx23c_jarredou.ckpt'
    download_file('https://github.com/jarredou/models/releases/download/DrumSep/drumsep_5stems_mdx23c_jarredou.ckpt')
    download_file('https://github.com/jarredou/models/releases/download/DrumSep/config_mdx23c.yaml')

elif model == 'DRUMSEP-MDX23C_DrumSep_6stem (by aufr33 & jarredou)':
    model_type = 'mdx23c'
    config_path = 'ckpts/aufr33-jarredou_DrumSep_model_mdx23c_ep_141_sdr_10.8059.yaml'
    start_check_point = 'ckpts/aufr33-jarredou_DrumSep_model_mdx23c_ep_141_sdr_10.8059.ckpt'
    download_file('https://github.com/jarredou/models/releases/download/aufr33-jarredou_MDX23C_DrumSep_model_v0.1/aufr33-jarredou_DrumSep_model_mdx23c_ep_141_sdr_10.8059.ckpt')
    download_file('https://github.com/jarredou/models/releases/download/aufr33-jarredou_MDX23C_DrumSep_model_v0.1/aufr33-jarredou_DrumSep_model_mdx23c_ep_141_sdr_10.8059.yaml')

elif model == '4STEMS-MelBand-Roformer_Large (by Amane)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/BS-Rofo-SW-Fixed.yaml'
    start_check_point = 'ckpts/BS-Rofo-SW-Fixed.ckpt'
    download_file('https://huggingface.co/jarredou/BS-ROFO-SW-Fixed/resolve/main/BS-Rofo-SW-Fixed.yaml')
    download_file('https://huggingface.co/jarredou/BS-ROFO-SW-Fixed/resolve/main/BS-Rofo-SW-Fixed.ckpt')

elif model == '6STEMS-BS-Roformer-SW':
    model_type = 'bs_roformer'
    config_path = 'ckpts/BS-Rofo-SW-Fixed.ckpt'
    start_check_point = 'ckpts/BS-Rofo-SW-Fixed.ckpt'
    download_file('https://huggingface.co/jarredou/BS-ROFO-SW-Fixed/resolve/main/BS-Rofo-SW-Fixed.yaml')
    download_file('https://huggingface.co/jarredou/BS-ROFO-SW-Fixed/resolve/main/BS-Rofo-SW-Fixed.ckpt')

elif model == '4STEMS-SCNet_XL_MUSDB18 (by ZFTurbo)':
    model_type = 'scnet'
    config_path = 'ckpts/config_musdb18_scnet_xl.yaml'
    start_check_point = 'ckpts/model_scnet_ep_54_sdr_9.8051.ckpt'
    download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v1.0.13/config_musdb18_scnet_xl.yaml')
    download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v1.0.13/model_scnet_ep_54_sdr_9.8051.ckpt')

elif model == '4STEMS-SCNet_Large (by starrytong)':
    model_type = 'scnet'
    config_path = 'ckpts/config_musdb18_scnet_large_starrytong.yaml'
    start_check_point = 'ckpts/SCNet-large_starrytong_fixed.ckpt'
    download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v1.0.9/config_musdb18_scnet_large_starrytong.yaml')
    download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v1.0.9/SCNet-large_starrytong_fixed.ckpt')

elif model == '4STEMS-SCNet_MUSDB18 (by starrytong)':
    model_type = 'scnet'
    config_path = 'ckpts/config_musdb18_scnet.yaml'
    start_check_point = 'ckpts/scnet_checkpoint_musdb18.ckpt'
    download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v.1.0.6/config_musdb18_scnet.yaml')
    download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v.1.0.6/scnet_checkpoint_musdb18.ckpt')

elif model == '4STEMS-BS-Roformer_MUSDB18 (by ZFTurbo)':
    model_type = 'bs_roformer'
    config_path = 'ckpts/config_bs_roformer_384_8_2_485100.yaml'
    start_check_point = 'ckpts/model_bs_roformer_ep_17_sdr_9.6568.ckpt'
    download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v1.0.12/config_bs_roformer_384_8_2_485100.yaml')
    download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v1.0.12/model_bs_roformer_ep_17_sdr_9.6568.ckpt')

elif model == 'DE-REVERB-MDX23C (by aufr33 & jarredou)':
    model_type = 'mdx23c'
    config_path = 'ckpts/config_dereverb_mdx23c.yaml'
    start_check_point = 'ckpts/dereverb_mdx23c_sdr_6.9096.ckpt'
    download_file('https://huggingface.co/jarredou/aufr33_jarredou_MDXv3_DeReverb/resolve/main/dereverb_mdx23c_sdr_6.9096.ckpt')
    download_file('https://huggingface.co/jarredou/aufr33_jarredou_MDXv3_DeReverb/resolve/main/config_dereverb_mdx23c.yaml')

elif model == 'DE-REVERB-MelBand-Roformer aggr./v2/19.1729 (by anvuew)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/dereverb_mel_band_roformer_anvuew.yaml'
    start_check_point = 'ckpts/dereverb_mel_band_roformer_anvuew_sdr_19.1729.ckpt'
    download_file('https://huggingface.co/anvuew/dereverb_mel_band_roformer/resolve/main/dereverb_mel_band_roformer_anvuew_sdr_19.1729.ckpt')
    download_file('https://huggingface.co/anvuew/dereverb_mel_band_roformer/resolve/main/dereverb_mel_band_roformer_anvuew.yaml')

elif model == 'DE-REVERB-BS-Roformer dereverb_room mono (by anvuew)':
    model_type = 'bs_roformer'
    config_path = 'ckpts/dereverb_room_anvuew.yaml'
    start_check_point = 'ckpts/dereverb_room_anvuew_sdr_13.7432.ckpt'
    download_file('https://huggingface.co/anvuew/dereverb_room/blob/main/dereverb_room_anvuew_sdr_13.7432.ckpt')
    download_file('https://huggingface.co/anvuew/dereverb_room/resolve/main/dereverb_room_anvuew.yaml')

elif model == 'DE-REVERB-MelBand-Roformer mono 20.4029 (by anvuew)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/dereverb_mel_band_roformer_anvuew.yaml'
    start_check_point = 'ckpts/dereverb_mel_band_roformer_mono_anvuew_sdr_20.4029.ckpt'
    download_file('https://huggingface.co/anvuew/dereverb_mel_band_roformer/resolve/main/dereverb_mel_band_roformer_mono_anvuew_sdr_20.4029.ckpt')
    download_file('https://huggingface.co/anvuew/dereverb_mel_band_roformer/resolve/main/dereverb_mel_band_roformer_anvuew.yaml')

elif model == 'DE-REVERB-Echo-MelBand-Roformer (by Sucial)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_dereverb-echo_mel_band_roformer.yaml'
    start_check_point = 'ckpts/dereverb-echo_mel_band_roformer_sdr_10.0169.ckpt'
    download_file('https://huggingface.co/Sucial/Dereverb-Echo_Mel_Band_Roformer/resolve/main/dereverb-echo_mel_band_roformer_sdr_10.0169.ckpt')
    download_file('https://huggingface.co/Sucial/Dereverb-Echo_Mel_Band_Roformer/resolve/main/config_dereverb-echo_mel_band_roformer.yaml')

elif model == 'LEAD-VOCAL-DE-REVERB-MelBand-Roformer (by Gabox)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/karaokegabox_1750911344.yaml'
    start_check_point = 'ckpts/Lead_VocalDereverb.ckpt'
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/experimental/Lead_VocalDereverb.ckpt')
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/karaoke/karaokegabox_1750911344.yaml')

elif model == 'DENOISE-MelBand-Roformer-1 (by aufr33)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/model_mel_band_roformer_denoise.yaml'
    start_check_point = 'ckpts/denoise_mel_band_roformer_aufr33_sdr_27.9959.ckpt'
    download_file('https://huggingface.co/jarredou/aufr33_MelBand_Denoise/resolve/main/denoise_mel_band_roformer_aufr33_sdr_27.9959.ckpt')
    download_file('https://huggingface.co/jarredou/aufr33_MelBand_Denoise/resolve/main/model_mel_band_roformer_denoise.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'DENOISE-MelBand-Roformer-2 (by aufr33)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/model_mel_band_roformer_denoise.yaml'
    start_check_point = 'ckpts/denoise_mel_band_roformer_aufr33_aggr_sdr_27.9768.ckpt'
    download_file('https://huggingface.co/jarredou/aufr33_MelBand_Denoise/resolve/main/denoise_mel_band_roformer_aufr33_aggr_sdr_27.9768.ckpt')
    download_file('https://huggingface.co/jarredou/aufr33_MelBand_Denoise/resolve/main/model_mel_band_roformer_denoise.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'DEBLEED-MelBand-Roformer (by unwa/97chris)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_bleed_suppressor_v1.yaml'
    start_check_point = 'ckpts/bleed_suppressor_v1.ckpt'
    download_file('https://huggingface.co/jarredou/bleed_suppressor_melband_rofo_by_unwa_97chris/resolve/main/bleed_suppressor_v1.ckpt')
    download_file('https://huggingface.co/jarredou/bleed_suppressor_melband_rofo_by_unwa_97chris/resolve/main/config_bleed_suppressor_v1.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'DENOISE-DEBLEED-MelBand-Roformer (by Gabox)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/inst_gabox.yaml'
    start_check_point = 'ckpts/denoisedebleed.ckpt'
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/instrumental/denoisedebleed.ckpt')
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/instrumental/inst_gabox.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-Male Female-BS-RoFormer Male Female Beta 7_2889 (by aufr33)':
    model_type = 'bs_roformer'
    config_path = 'ckpts/config_chorus_male_female_bs_roformer.yaml'
    start_check_point = 'ckpts/bs_roformer_male_female_by_aufr33_sdr_7.2889.ckpt'
    download_file('https://huggingface.co/RareSirMix/AIModelRehosting/resolve/main/bs_roformer_male_female_by_aufr33_sdr_7.2889.ckpt')
    download_file('https://huggingface.co/Sucial/Chorus_Male_Female_BS_Roformer/resolve/main/config_chorus_male_female_bs_roformer.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'PHANTOM-CENTER-HTDemucs (by wesleyr36)':
    model_type = 'htdemucs'
    config_path = 'ckpts/config_htdemucs_similarity.yaml'
    start_check_point = 'ckpts/model_htdemucs_ep_21_sdr_13.6970.ckpt'
    download_file('https://huggingface.co/jarredou/HTDemucs_Similarity_Extractor_by_wesleyr36/resolve/main/model_htdemucs_ep_21_sdr_13.6970.ckpt')
    download_file('https://huggingface.co/jarredou/HTDemucs_Similarity_Extractor_by_wesleyr36/resolve/main/config_htdemucs_similarity.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'GUITAR-MelBand-Roformer (by becruily)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_guitar_becruily.yaml'
    start_check_point = 'ckpts/becruily_guitar.ckpt'
    download_file('https://huggingface.co/becruily/mel-band-roformer-guitar/resolve/main/becruily_guitar.ckpt')
    download_file('https://huggingface.co/becruily/mel-band-roformer-guitar/resolve/main/config_guitar_becruily.yaml')
    conf_edit(config_path, chunk_size, overlap)


!python inference.py \
    --model_type {model_type} \
    --config_path '{config_path}' \
    --start_check_point '{start_check_point}' \
    --input_folder '{input_folder}' \
    --store_dir '{output_folder}' \
    {('--extract_instrumental' if extract_instrumental else '')} \
    {('--flac_file' if flac_file else '')} \
    {('--use_tta' if use_tta else '')} \
    {('--pcm_type ' + pcm_type if pcm_type else '')}

**INST-Mel-Roformers like v1/1x/2** have switched output file names - files labelled as vocals are instrumentals <br>
But e.g. not v1e - so if you uncheck extract_instrumentals for it, only one stem called "other" will be rendered, and it will be instrumental.<br><br>
**Mel Karaoke by becruily and Duality** models output 2 stems, and don't need "**extract_instrumental**" option enabled (inverted stem will rather have worse quality, than the model output, plus you won't end up with 3 output files).
<br><br>
**TTA** - results in longer separation time, "it gives a little better SDR score but hard to tell if it's really audible in most cases". <br> it “means "test time augmentation", (...) it will do 3 passes on the audio file instead of 1. 1 pass with be with original audio. 1 will be with inverted stereo (L becomes R, R become L). 1 will be with phase inverted and then results are averaged for final output. ” - jarredou
<br><br>
**Overlap** - higher means longer separation time. 4 is already balanced value, 2 is fast and some people still won't notice any difference. Normally there's not point going over 8.<br><br>
**Chunk_size** - most models use the default 485100 (besides Beta6X - 529200, 6 stems SW - 588800 [882000 will probably crash on free T4], and Amane's 4 stems Large - 661500), and achieves the highest SDR with this value (higher than training chunk), but some people occasionally use also 112455 to get different results.<br><br>

If your separation can't start and "**Total files found: 0**" is shown, be aware that: <br>1) Input must be a path to a folder containing audio files, not direct path to an audio file<br> 2) The Colab is case aware - e.g. call your folder "input" not "Input".<br> 3) Check if your Google Drive mounting was executed correctly. Open file manager on the left to check if your drive folder is not empty. If it's the case, force remount with the following line:

In [ ]:
drive.mount("/content/drive", force_remount=True)

4) Consider uploading your files to input folder on GDrive before running the Colab - rarely it may happen that the files in the file manager might be invisible despite refreshing the files view - then you can launch the cell above 2 or 3 times to fix it.
<br>

**"Propagation unsuccessful" error** - click on connect again button in the right top corner (might happen if you delete the envrinment manually and start from the first cell again).